In [8]:
import os

# Define the path where the dataset was extracted
extracted_dir_path = r"D:\BITNBUILD2024\food101"  # Update this to your actual path

# Now construct the train data directory path
train_data_dir = os.path.join(extracted_dir_path, 'images')


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the extracted directory
extracted_dir_path = r"D:\BITNBUILD2024\food101"  # Update this with your actual path

# Directory containing the images
train_data_dir = os.path.join(extracted_dir_path, 'images')

# Image dimensions and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

# Create an ImageDataGenerator for training and validation data
datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values
    validation_split=0.2         # Use 20% of data for validation
)

# Load training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'            # Training subset
)

# Load validation data
val_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'          # Validation subset
)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load the MobileNetV2 model without the top layer (use imagenet weights)
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Number of food categories
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 328s 35us/step


In [11]:
EPOCHS = 10

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=val_generator.samples // BATCH_SIZE
)


Epoch 1/10


C:\Users\Max Gonsalves\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1886s 744ms/step - accuracy: 0.3016 - loss: 2.9532 - val_accuracy: 0.5029 - val_loss: 1.9800
Epoch 2/10


C:\Program Files\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1s 202us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 2.1876
Epoch 3/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1221s 483ms/step - accuracy: 0.4553 - loss: 2.1531 - val_accuracy: 0.5158 - val_loss: 1.9302
Epoch 4/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 0s 84us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 1.3403
Epoch 5/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1383s 547ms/step - accuracy: 0.4764 - loss: 2.0516 - val_accuracy: 0.5246 - val_loss: 1.8873
Epoch 6/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 0s 68us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 2.2361
Epoch 7/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1247s 493ms/step - accuracy: 0.5006 - loss: 1.9519 - val_accuracy: 0.5270 - val_loss: 1.8932
Epoch 8/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 2.1140
Epoch 9/10
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 953s 37

In [12]:
model.save('food101_mobilenetv2.h5')